In [1]:
import numpy as np
import time
import sys
import os
import tqdm

import torch
from torch import nn
import torch.nn.functional as F


from catr.configuration import Config
from catr.models.utils import NestedTensor, nested_tensor_from_tensor_list, get_rank
from catr.models.backbone import build_backbone
from catr.models.transformer import build_transformer
from catr.models.position_encoding import PositionEmbeddingSine
from catr.models.caption import MLP

import json

from dataset.dataset import ImageFeatureDataset
from torch.utils.data import DataLoader
from transformer_ethan import *

In [2]:
words = np.load("glove_embed.npy")
with open('word2ind.json') as json_file: 
    word2ind = json.load(json_file) 
with open('ind2word.json') as json_file: 
    ind2word = json.load(json_file) 
config = Config()
config.device = 'cpu' # if running without GPU
config.feature_dim = 1024
config.pad_token_id = word2ind["<S>"]
config.hidden_dim = 300
config.nheads = 10
config.batch_size = 1
config.vocab_size = words.shape[0]
config.__dict__["pre_embed"] = torch.from_numpy(words)

In [3]:
model, criterion = main(config)
device = torch.device(config.device)
model.to(device)

Initializing Device: cpu
Number of params: 33908144


Xray_Captioner(
  (input_proj): Conv2d(1024, 300, kernel_size=(1, 1), stride=(1, 1))
  (position_embedding): PositionEmbeddingSine()
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=300, out_features=300, bias=True)
          )
          (linear1): Linear(in_features=300, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=300, bias=True)
          (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=3

In [4]:
param_dicts = [
        {"params": [p for n, p in model.named_parameters(
        ) if "backbone" not in n and p.requires_grad]},
        {
            "params": [p for n, p in model.named_parameters() if "backbone" in n and p.requires_grad],
            "lr": config.lr_backbone,
        },
    ]

In [5]:
optimizer = torch.optim.AdamW(
        param_dicts, lr=config.lr, weight_decay=config.weight_decay)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, config.lr_drop)

In [6]:
dataset_train = ImageFeatureDataset('../mimic_features/paths.csv',
                             '../mimic_features/')
#dataset_val = ImageFeatureDataset('../mimic_features/paths.csv',
                           #  '../mimic_features/', 'val') # this would be ideal edward
#sampler_train = torch.utils.data.RandomSampler(dataset_train)
#sampler_val = torch.utils.data.SequentialSampler(dataset_val)

#batch_sampler_train = torch.utils.data.BatchSampler(
#        sampler_train, config.batch_size, drop_last=True
#    )

data_loader_train = DataLoader(dataset_train, batch_size=1, shuffle=False)
# Eddie get data loader to take: DataLoader(
        #dataset_train, batch_sampler=batch_sampler_train, num_workers=config.num_workers)
    #data_loader_val = DataLoader(dataset_val, config.batch_size,
    #                             sampler=sampler_val, drop_last=False, num_workers=config.num_workers)
print(f"Train: {len(dataset_train)}")

Train: 10


In [7]:
if os.path.exists(config.checkpoint):
    print("Loading Checkpoint...")
    checkpoint = torch.load(config.checkpoint, map_location='cpu')
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
    config.start_epoch = checkpoint['epoch'] + 1

print("Start Training..")

Start Training..


In [8]:
def train_one_epoch(model, criterion, data_loader,
                    optimizer, device, max_norm):
    model.train()
    criterion.train()

    epoch_loss = 0.0
    total = len(data_loader)

    with tqdm.tqdm(total=total) as pbar:
        for images, masks, caps, cap_masks in data_loader:
            samples = utils.NestedTensor(images, masks).to(device)
            caps = caps.to(device)
            cap_masks = cap_masks.to(device)

            outputs = model(samples, caps[:, :-1], cap_masks[:, :-1])
            loss = criterion(outputs.permute(0, 2, 1), caps[:, 1:])
            loss_value = loss.item()
            epoch_loss += loss_value

            if not math.isfinite(loss_value):
                print(f'Loss is {loss_value}, stopping training')
                sys.exit(1)

            optimizer.zero_grad()
            loss.backward()
            if max_norm > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
            optimizer.step()

            pbar.update(1)

    return epoch_loss / total

In [9]:
for epoch in range(config.start_epoch, config.epochs):
        print(f"Epoch: {epoch}")
        epoch_loss = train_one_epoch(
            model, criterion, data_loader_train, optimizer, device, config.clip_max_norm)
        lr_scheduler.step()
        print(f"Training Loss: {epoch_loss}")

        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'epoch': epoch,
        }, config.checkpoint)

        #validation_loss = evaluate(model, criterion, data_loader_val, device)
        #print(f"Validation Loss: {validation_loss}")

        print()

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0


ValueError: not enough values to unpack (expected 4, got 2)

In [10]:
iterations = iter(data_loader_train)

In [12]:
im, note = next(iterations)

In [15]:
note

[('<S>',),
 ('No',),
 ('evidence',),
 ('of',),
 ('consolidation',),
 ('to',),
 ('suggest',),
 ('pneumonia',),
 ('is',),
 ('seen',),
 ('.',),
 ('<s>',),
 ('There',),
 ('is',),
 ('some',),
 ('retrocardiac',),
 ('atelectasis',),
 ('.',),
 ('<s>',),
 ('A',),
 ('small',),
 ('left',),
 ('pleural',),
 ('effusion',),
 ('may',),
 ('be',),
 ('present',),
 ('.',),
 ('<s>',),
 ('No',),
 ('pneumothorax',),
 ('is',),
 ('seen',),
 ('.',),
 ('<s>',),
 ('No',),
 ('pulmonary',),
 ('edema',),
 ('.',),
 ('<s>',),
 ('A',),
 ('right',),
 ('granuloma',),
 ('is',),
 ('unchanged',),
 ('.',),
 ('<s>',),
 ('The',),
 ('heart',),
 ('is',),
 ('mildly',),
 ('enlarged',),
 (',',),
 ('unchanged',),
 ('.',),
 ('<s>',),
 ('There',),
 ('is',),
 ('tortuosity',),
 ('of',),
 ('the',),
 ('aorta',),
 ('.',),
 ('<s>',),
 ('</s>',)]